In [5]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

from SimulateDay import *


def calculate_percentage(fee, SalePrice, stock):
    if SalePrice == 0:
        return 0
    percentage = (fee / SalePrice) * 100
    portfolio = pd.read_csv('portfolio.csv')
    
    # Update the SalePrice for the given stock on the most recent day
    portfolio.loc[(portfolio['Stock Name'] == stock) & (portfolio['Day'] == portfolio['Day'].max()), 'SalePrice'] = SalePrice - fee
    
    # Write to the log file in append mode
    with open('Log.txt', 'a') as file:
        file.write(f"{stock}, sold at {SalePrice - fee}, fee: {fee}, percentage: {percentage}\n")
    
    # Print the log message
    print(f"{stock}, sold at {SalePrice - fee}, fee: {fee}, percentage: {percentage}")
    
    return portfolio

# Example usage
fee = 0.02
SalePrice = 124.13
stock = "AMZN"
Sale = calculate_percentage(fee, SalePrice, stock).fillna(0)

AMZN, sold at 124.11, fee: 0.02, percentage: 0.016112140497865142


In [19]:
def execute_buy(stock_name, purchase_price):
    portfolio = pd.read_csv('portfolio.csv')
    stock = portfolio[(portfolio['Stock Name'] == stock_name) & (portfolio['Day'] == portfolio['Day'].max())]
    
    if len(stock) == 0:
        # Fetch stock data
        stock_data = get_stock_data(stock_name)
        stock_price = stock_data['Close'].iloc[-1]
        
        # Calculate the number of shares that can be bought
        shares = purchase_price / stock_price
        
        # Append the new transaction to the portfolio
        portfolio = portfolio.append({
            'Stock Name': stock_name,
            'Day': portfolio['Day'].max() + 1,
            'Action': 'Buy',
            'Stock Price': stock_price,
            'Cash': purchase_price - (shares * stock_price),
            'Shares Held': shares,
            'Portfolio Value': shares * stock_price,
            'Date': stock_data['Date'].iloc[-1],
            'Actual Sell': 0
        }, ignore_index=True)
    else:
        stock_price = stock['Stock Price'].values[0]
        shares_to_buy = purchase_price / stock_price
        shares = stock['Shares Held'].values[0] + shares_to_buy
        portfolio.loc[(portfolio['Stock Name'] == stock_name) & (portfolio['Day'] == portfolio['Day'].max()), 'Shares Held'] += shares_to_buy 
        portfolio.loc[(portfolio['Stock Name'] == stock_name) & (portfolio['Day'] == portfolio['Day'].max()), 'Actual Buy'] = True
        portfolio.loc[(portfolio['Stock Name'] == stock_name) & (portfolio['Day'] == portfolio['Day'].max()), 'Portfolio Value'] = (shares * stock_price)

    portfolio.to_csv('portfolio.csv', index=False)
    
    return portfolio

purchase_price = 20
stock_name = "INTC"
portfolio = execute_buy(stock_name, purchase_price)
portfolio.tail(7)

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date,Actual Sell,Actual Buy
42,AAPL,6,Hold,220.679993,0,0.331895,73.242613,2024-11-04,0.0,NaN
43,XOM,6,Buy,118.294998,0,0.811577,96.005524,2024-11-04,0.0,True
44,AMZN,6,Buy,194.729996,0,0.746501,145.366042,2024-11-04,0.0,True
45,LYV,6,Hold,117.099998,0,0.421621,49.371784,2024-11-04,0.0,NaN
46,INTC,6,Buy,22.205000,0,2.995483,66.514697,2024-11-04,0.0,True
47,DNMR,6,Hold,0.301600,0,1.820000,0.548912,2024-11-04,0.0,NaN
48,SPCE,6,Hold,6.400000,0,0.246548,1.577909,2024-11-04,0.0,NaN


In [10]:
ap = pd.read_csv('portfolio.csv')
ap.tail(7)

,Stock Name,Day,Action,Stock Price,Cash,Shares Held,Portfolio Value,Date,Actual Sell
42,AAPL,6,Hold,220.679993,0,0.331895,73.242613,2024-11-04,0.0
43,XOM,6,Buy,118.294998,0,0.642508,76.005524,2024-11-04,0.0
44,AMZN,6,Buy,194.729996,0,0.643794,125.366042,2024-11-04,0.0
45,LYV,6,Hold,117.099998,0,0.421621,49.371784,2024-11-04,0.0
46,INTC,6,Buy,22.205000,0,2.094785,46.514697,2024-11-04,0.0
47,DNMR,6,Hold,0.301600,0,1.820000,0.548912,2024-11-04,0.0
48,SPCE,6,Hold,6.400000,0,0.246548,1.577909,2024-11-04,0.0


In [18]:
20/22.205000

0.900698040981761